In [121]:
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split

import math
import numpy as np
import argparse
import glob
import cv2
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split
from __future__ import division
from math import cos, sin

In [122]:
image = cv2.imread("D:/coin.jpg", 1)

In [123]:
d = 1024 / image.shape[1]
dim = (1024, int(image.shape[0] * d))
image = cv2.resize(image, dim, interpolation=cv2.INTER_AREA)
cv2.imshow("input", image)
cv2.waitKey(0)
cv2.destroyAllWindows()


In [124]:
output = image.copy()

In [125]:
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

In [126]:
clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
gray = clahe.apply(gray)

In [127]:
def calcHistogram(img):
    # create mask
    m = np.zeros(img.shape[:2], dtype="uint8")
    (w, h) = (int(img.shape[1] / 2), int(img.shape[0] / 2))
    cv2.circle(m, (w, h), 60, 255, -1)
    h = cv2.calcHist([img], [0, 1, 2], m, [8, 8, 8], [0, 256, 0, 256, 0, 256])
    return cv2.normalize(h, h).flatten()

In [128]:
def calcHistFromFile(file):
    img = cv2.imread(file)
    return calcHistogram(img)

In [129]:
class Enum(tuple): __getattr__ = tuple.index

In [130]:
Material = Enum(('Copper', 'Brass', 'Euro1', 'Euro2'))

In [131]:
sample_images_copper = glob.glob("D:/Masterfile/sample_images/copper/*")
sample_images_brass = glob.glob("D:/Masterfile/sample_images/brass/*")
sample_images_euro1 = glob.glob("D:/Masterfile/sample_images/euro1/*")
sample_images_euro2 = glob.glob("D:/Masterfile/sample_images/euro2/*")

In [132]:
X = []
y = []
for i in sample_images_copper:
    X.append(calcHistFromFile(i))
    y.append(Material.Copper)
for i in sample_images_brass:
    X.append(calcHistFromFile(i))
    y.append(Material.Brass)
for i in sample_images_euro1:
    X.append(calcHistFromFile(i))
    y.append(Material.Euro1)
for i in sample_images_euro2:
    X.append(calcHistFromFile(i))
    y.append(Material.Euro2)

In [133]:
clf = MLPClassifier(solver="lbfgs")

In [134]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=.4)

In [135]:
clf.fit(X_train, y_train)
score = int(clf.score(X_test, y_test) * 100)
print("Classifier mean accuracy: ", score)

Classifier mean accuracy:  94


In [136]:
blurred = cv2.GaussianBlur(gray, (7, 7), 0)

In [137]:
circles = cv2.HoughCircles(blurred, cv2.HOUGH_GRADIENT, dp=2.2, minDist=100,
                           param1=200, param2=100, minRadius=50, maxRadius=120)

In [138]:

def predictMaterial(roi): 
    hist = calcHistogram(roi)
    s = clf.predict([hist])
    return Material[int(s)]



diameter = []
materials = []
coordinates = []

count = 0
if circles is not None:
    for (x, y, r) in circles[0, :]:
        diameter.append(r)

    circles = np.round(circles[0, :]).astype("int")
    for (x, y, d) in circles:
        count += 1
        coordinates.append((x, y))

        roi = image[y - d:y + d, x - d:x + d]
        material = predictMaterial(roi)
        materials.append(material)

        if False:
            m = np.zeros(roi.shape[:2], dtype="uint8")
            w = int(roi.shape[1] / 2)
            h = int(roi.shape[0] / 2)
            cv2.circle(m, (w, h), d, (255), -1)
            maskedCoin = cv2.bitwise_and(roi, roi, mask=m)
        

        cv2.circle(output, (x, y), d, (0, 255, 0), 2)
        cv2.putText(output, material,
                    (x - 40, y), cv2.FONT_HERSHEY_PLAIN,
                    1.5, (0, 255, 0), thickness=2, lineType=cv2.LINE_AA)

In [139]:
d = 500 / output.shape[1]  
dim = (500, int(output.shape[0] * d))
image = cv2.resize(image, dim, interpolation=cv2.INTER_AREA)
output = cv2.resize(output, dim, interpolation=cv2.INTER_AREA)

In [140]:
cv2.imshow("Output",(output))
cv2.waitKey()
cv2.destroyAllWindows()
image2=cv2.imread("output.jpg",1)
print('done')

done


In [141]:
cv2.imwrite("D:/Coindetected.jpg", output) 


True